In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np
import csv
import sympy
from numpy import array

In [2]:

# wp_E = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='p_E', continuous_update=False) # probability that RP runs correctly
# wroi = widgets.FloatSlider(min=0,max=1,step=.01,value=.5,description='roi', continuous_update=False) # what percent of reward is profit
# winsurance = widgets.FloatSlider(min=0,max=1,step=.01,value=0 ,description='I',continuous_update=False) # % of deposit that can be recovered because execution is so cheap.
# wC_f= widgets.FloatSlider(min=0,max=1000,step=.1,value=0, description='C_f',continuous_update=False) # cost for RP to be fraudulant 

# matchPrice = 1
# mediatorPrice = 1


def RPU(n,PR,p_a,pi,p_v=1,C_f=0,p_E=1, ec=0):
    p2 = 1-p_a
#     D = pi * PR
#     D = pi * PR + pi*n
    D = pi * PR + (pi * n)
    
    
    execCost = ec

    #Cj is cost of execution for fraudulant job, picking a random number or "close" value
    fraudCost = C_f
    
    JC_HRP = p_v*p_E*p_a*(pi-execCost) #compute answer 1 (p_a) correctly (p_E) receive reward (pi)
    
    NDJC_HRP_Gain = p_v*p_E*p2*(1-p_a**n)*(pi-execCost) #compute answer 2 (p2) correctly (p_E) get sent to mediation, who finds nondeterminism (1-p_a^n) and recieve reward (r) and nonDet bonus (f2-f)*k kickback=k
    NDJC_HRP_Lose = p_v*p_E*p2*(p_a**n)*(-D-execCost) # compute answer 2 correctly (p2) get sent to mediation, who doesn't find nondeterminsm (p_a^n) and get fined (f)
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose 
    
    DRP_Gain = p_v*(1-p_E)*(1-p_a**n)*(pi-fraudCost)# compute incorrectly (1-p_E) get sent to mediation, finds nondeterminism (1-p_a^n) get compensation.
    DRP_Lose = p_v*(1-p_E)*(p_a**n)*(-D-fraudCost) #compute incorrectly (1-p_E) get sent to mediation, who doesn't find nondeterminsm (p_a^n) and get fined (f)
    DRP = DRP_Gain+DRP_Lose
    
#     DRP_scared  = p_v*(1-p_E)*(pi-fraudCost) # JC doesn't want to risk getting caught, just pay RP. 
    
    #     noVerifypayout = (1-p_v)*pi #compute answer 1 (p_a) correctly (p_E) receive reward (pi)
    noID_HRP = (1-p_v) * p_E * (pi - execCost)
    noID_DRP = (1-p_v) * (1-p_E) * (pi - fraudCost)
    noID = noID_HRP + noID_DRP
    
    
    RCnonDet = JC_HRP + ND + DRP + noID
    return RCnonDet


In [3]:
rn,rPR,rp_a,rpi,rp_v,rC_f,rp_E, rec, = sympy.symbols('n PR p_a pi p_v C_f p_E ec')

RU = RPU(rn,rPR,rp_a,rpi,rp_v,rC_f,rp_E, rec)
display(RU)

p_E*p_a*p_v*(-ec + pi) + p_E*p_a**n*p_v*(1 - p_a)*(-PR*pi - ec - n*pi) + p_E*p_v*(1 - p_a)*(1 - p_a**n)*(-ec + pi) + p_E*(1 - p_v)*(-ec + pi) + p_a**n*p_v*(1 - p_E)*(-C_f - PR*pi - n*pi) + p_v*(1 - p_E)*(1 - p_a**n)*(-C_f + pi) + (1 - p_E)*(1 - p_v)*(-C_f + pi)

In [4]:
LRU = RU.subs({rp_a:1 })
display(LRU)

p_E*p_v*(-ec + pi) + p_E*(1 - p_v)*(-ec + pi) + p_v*(1 - p_E)*(-C_f - PR*pi - n*pi) + (1 - p_E)*(1 - p_v)*(-C_f + pi)

In [20]:
dLRU = sympy.diff(LRU,rp_E)
display(dLRU)
dLRU = sympy.simplify(dLRU)
display(dLRU)
dLRU = sympy.collect(dLRU,rp_v)
display(dLRU)
dLRU = sympy.collect(dLRU,rpi)
display(dLRU)

p_v*(-ec + pi) - p_v*(-C_f - PR*pi - n*pi) + (1 - p_v)*(-ec + pi) + (-C_f + pi)*(p_v - 1)

C_f + PR*p_v*pi - ec + n*p_v*pi + p_v*pi

C_f - ec + p_v*(PR*pi + n*pi + pi)

C_f - ec + p_v*pi*(PR + n + 1)

In [36]:
sol_dLRU_pv = sympy.solve([dLRU], [rp_v])
sol_dLRU_pv[rp_v]

(-C_f + ec)/(pi*(PR + n + 1))

In [41]:
points = 51
x,step = np.linspace(0,1,points,retstep=True)
print("step size: %s" %step)

        #-M-     -JC-                -RP-
def plot(n,PR,   pi,p_v,       C_f,p_E,ec):
    
    
    yLRU = RPU(n=n,PR=PR,     p_a=np.ones(points),pi=pi,p_v=p_v,        C_f=C_f,p_E=x,ec=ec)
    
    
#     yLJU = list(map(lambda v: float(LJU.subs({sC_v:C_v, sB:B, sp_v:v, spi:pi, sPR:PR,  sp_E:p_E  })),x))
    print(max(yLRU))
    
    plt.plot(x,yLRU, label="LRU")
    plt.xlabel('p_E')
    plt.ylabel('Util') 
    plt.legend()
    plt.grid(visible=True)
    plt.show
    

step size: 0.02


In [42]:
wn = widgets.IntSlider(min=0,max=10,step=1,value=1, description='n',continuous_update=False)
wPR = widgets.FloatSlider(min=0,max=100,step=.5,value=1, description='PR', continuous_update=False) 

wpi = widgets.IntSlider(min=0,max=1000,step=1,value=1, description='pi', continuous_update=False)
wp_v = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='p_v', continuous_update=False) # how often JC verifies

wC_f =widgets.FloatSlider(min=0,max=100,step=1,value=0, description='C_F', continuous_update=False)
wp_E = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='p_E', continuous_update=False) # probability that RP runs correctly
wec = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='ec', continuous_update=False) #Penalty Rate


ui1 = widgets.HBox([wn,wPR])
ui2 = widgets.HBox([wpi,wp_v])
ui3 = widgets.HBox([wC_f,wp_E,wec])

# interactive_plot = widgets.interactive(plot, n=wn,PR=wPR,NDPR=wNDPR,pi=wpi,p_v=wp_v,cj=wcj, p_E=wp_E,roi=wroi,Insurance=winsurance);
interactive_plotFinal = widgets.interactive_output(plot,{'n':wn,'PR':wPR,   'pi':wpi,'p_v':wp_v,    'C_f':wC_f,'p_E':wp_E,'ec':wec});

# interactive_plotFinal.layout.height = '600px'

display(interactive_plotFinal,ui1,ui2,ui3)

Output()